In [0]:
# Este código se ejecuta en una celda de un Notebook de Databricks
# Su responsabilidad es tomar el archivo CSV de stock y cargarlo en una tabla Delta de la capa Bronce.
# Esta versión es robusta para manejar datos complejos y nombres de columna inválidos.

import re

# 1. --- Configuración de Rutas y Tablas ---
volume_path = "/Volumes/workspace/tecnomundo_data_raw/uploads_stock/"
csv_file_name = "Stock.csv"
source_csv_path = f"{volume_path}{csv_file_name}"
output_bronze_table = "workspace.tecnomundo_data_raw.stock_raw"

print(f"Ruta del archivo CSV de origen: {source_csv_path}")
print(f"Tabla de Destino (Bronce): {output_bronze_table}")

# 2. --- Lectura Robusta del CSV ---
print(f"\nLeyendo archivo CSV desde '{source_csv_path}' con opciones robustas...")
try:
    # Usamos opciones avanzadas para prevenir la corrupción de datos:
    # - inferSchema: Permite a Spark detectar los tipos de datos (ej. Integer, String).
    # - multiLine: Permite que los campos de texto contengan saltos de línea.
    # - escape: Define el carácter de escape para las comillas dentro de los campos.
    df_csv = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("multiLine", "true") \
        .option("escape", "\"") \
        .load(source_csv_path)

    print(f"Lectura exitosa. Se encontraron {df_csv.count()} registros.")
    print("Esquema inferido por Spark con nombres de columna originales:")
    df_csv.printSchema()

    # 3. --- Limpieza de Nombres de Columna ---
    print("\nLimpiando nombres de columna para que sean compatibles con Delta...")
    
    # Creamos una función para limpiar cada nombre de columna
    def sanitize_column_name(col_name):
        # Reemplaza caracteres no alfanuméricos (excepto '_') con un guion bajo
        clean_name = re.sub(r'[^a-zA-Z0-9_]', '_', col_name)
        # Convierte a minúsculas
        return clean_name.lower()

    # Aplicamos la limpieza a todas las columnas del DataFrame
    cleaned_columns = [sanitize_column_name(c) for c in df_csv.columns]
    df_cleaned = df_csv.toDF(*cleaned_columns)

    print("Nombres de columna limpios:")
    df_cleaned.printSchema()
    
    # 4. --- Carga a la Tabla Bronce ---
    # Guardamos el DataFrame con los nombres de columna limpios.
    (df_cleaned.write
     .mode("overwrite")
     .option("overwriteSchema", "true")
     .saveAsTable(output_bronze_table))
    
    print(f"\n¡Tabla '{output_bronze_table}' creada/actualizada exitosamente!")

except Exception as e:
    print(f"Error durante la ingesta a la capa Bronce: {e}")
    dbutils.notebook.exit("Fallo en el proceso de ingesta a Bronce.")

# 5. --- Verificación Final ---
print("\nMostrando una muestra de la tabla Bronce creada:")
display(spark.table(output_bronze_table))


In [0]:
from pyspark.sql.functions import col

bronze_df = spark.table("workspace.tecnomundo_data_raw.stock_raw")
non_numeric_stock = bronze_df.filter(~col("stock").cast("double").isNotNull())
display(non_numeric_stock.select("stock"))